In [59]:
# 初始化参数
import os
path = '/project'
# alpha = 'abcdefghijklmnopqrstuvwxyz'
# alpha = '0123456789+-÷×=)('
# alpha = '0123456789.'
alpha = '√×'
imgH = 32
imgC = 3
nh = 256
batch_size = 100
thread_accuracy = 0.995

def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)
        

def valid_net(net, valid_dataset, criterion):
    
    valid_loader = torch.utils.data.DataLoader(
        valid_dataset, batch_size=batch_size,
        shuffle=True,
        collate_fn=dataset.adjustCollate(imgH=32, keep_ratio=False))   
    i = 0
    n_correct = 0    
    net.eval()
    with torch.no_grad():
        for step, values in enumerate(valid_loader):
            images = Variable(values[0].type(dtype), requires_grad=False)
            labels = values[1]
            t, l = converter.encode(labels)
            utils.loadData(text, t)
            utils.loadData(length, l)

            preds = net(images).cpu()
            preds_size = Variable(torch.IntTensor([preds.size(0)] * preds.size(1)))

            _, preds = preds.max(2)
            preds = preds.transpose(1, 0).contiguous().view(-1)
            sim_preds = converter.decode(preds.data, preds_size.data, raw=False)

            for pred, target in zip(sim_preds, labels):
                if pred == target.lower():
                    n_correct += 1        
    print('sim preds -->{}'.format(sim_preds))
    print('text -->{}'.format(labels))
    accuracy = n_correct / float(len(valid_dataset))
    return accuracy, n_correct
        
print('over !!!')


over !!!


In [60]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from lib.crnn import CRNNClassify
from torch.autograd import Variable
import lib.dataset as dataset
import lib.utils as utils
import time
import importlib
importlib.reload(dataset)
importlib.reload(utils)

net = CRNNClassify(imgH=imgH,nc=imgC,nh=nh, nclass=len(alpha)+1)
net.apply(weights_init)

# net = torch.load('{}/acrnn_best.pkl'.format(path))

converter = utils.strLabelConverter(alpha)

text = torch.IntTensor(batch_size * 5)
length = torch.IntTensor(batch_size)

text = Variable(text,requires_grad=False)
length = Variable(length,requires_grad=False)

train_dataset = dataset.lmdbDataset(root=os.path.join(path,'data','lmdb'))
print('train data set length -->{}'.format(len(train_dataset)))
train_set, val_set = torch.utils.data.random_split(train_dataset,
                                                   [int(0.9*len(train_dataset)), 
                                                    int(0.1*len(train_dataset))])
assert train_dataset

train_loader = torch.utils.data.DataLoader(
    train_set, batch_size=batch_size,
    shuffle=True,
    collate_fn=dataset.adjustCollate(imgH=32, keep_ratio=False))

if torch.cuda.is_available():
    net.cuda()
    dtype = torch.cuda.FloatTensor
    print("Using GPU")
else:
    dtype = torch.FloatTensor
    print("No GPU detected")

optimizer = torch.optim.Adam(net.parameters(), lr=1e-4,betas=(0.5, 0.999))
# optimizer = torch.optim.RMSprop(net.parameters(), lr=1e-2)
criterion = nn.CTCLoss()
lowest_loss = float('inf')

current_accuracy = 0.0
start_time = time.time()
for epoch in range(10000):
    net.train()
    loss = None
    
    sum_loss = 0.0
    total_step = 0    
    for step, values in enumerate(train_loader):
        images = Variable(values[0].type(dtype), requires_grad=False)
        labels = values[1]
        t, l = converter.encode(labels)
        utils.loadData(text, t)
        utils.loadData(length, l)
        
        
        preds = net(images).cpu()
        preds_size = Variable(torch.IntTensor([preds.size(0)] * preds.size(1)))
        
        loss = criterion(preds, text, preds_size, length) / preds.size(1)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()        
        
    if epoch % 2 == 0:
        accuracy, num_correct = valid_net(net,val_set,loss)
        if accuracy > current_accuracy:
            current_accuracy = accuracy
            torch.save(net,'{}/save/scrnn_{}.pkl'.format(path,epoch))
            torch.save(net.state_dict(), '{}/save/scrnn_{}.pkh'.format(path,epoch))
        
        print('epoch --> {} loss -->{} accuracy -->{} correct --> {}, time {}'.format(epoch, loss,accuracy,num_correct,(time.time()-start_time))) 
        start_time = time.time()
        if current_accuracy >= thread_accuracy:
            break
       

print('over ..............')



train data set length -->100000
Using GPU
sim preds -->['×', '×', '×', '√', '×', '√', '√', '√', '×', '√', '√', '√', '√', '×', '×', '×', '√', '√', '√', '√', '√', '×', '×', '×', '×', '×', '√', '×', '√', '√', '√', '×', '√', '×', '√', '√', '×', '√', '√', '×', '√', '√', '×', '√', '√', '×', '×', '×', '×', '×', '×', '√', '×', '×', '√', '√', '×', '√', '×', '√', '×', '√', '×', '√', '×', '√', '√', '×', '√', '√', '×', '×', '√', '√', '√', '×', '√', '×', '×', '√', '×', '√', '√', '×', '×', '√', '×', '×', '√', '√', '×', '√', '√', '√', '√', '×', '×', '√', '√', '×']
text -->('×', '×', '×', '√', '×', '√', '√', '√', '×', '√', '√', '√', '√', '×', '×', '×', '√', '√', '√', '√', '√', '×', '×', '×', '×', '×', '√', '×', '√', '√', '√', '×', '√', '×', '√', '√', '×', '√', '√', '×', '√', '√', '×', '√', '√', '×', '×', '×', '×', '×', '×', '√', '×', '×', '√', '√', '×', '√', '×', '√', '×', '√', '×', '√', '×', '√', '√', '×', '√', '√', '×', '×', '√', '√', '√', '×', '√', '×', '×', '√', '×', '√', '√', '×', '×', '√', '×', 